<a href="https://colab.research.google.com/github/Vitormdz/Projeto-da-Imers-o-IA-Alura-Google/blob/main/Projeto_Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_Aula_05_Agentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)
# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura não tem uma data fixa para a Imersão IA com Google Gemini. A melhor forma de saber quando a próxima edição acontecerá é:

*   **Acompanhar as redes sociais da Alura:** Eles costumam anunciar novas edições em seus perfis no Instagram, LinkedIn, Twitter e Facebook.
*   **Ficar de olho no site da Alura:** A página da Imersão IA com Google Gemini (se houver uma ativa) ou a página de eventos da Alura geralmente terá as informações sobre próximas edições.
*   **Assinar a newsletter da Alura:** Assim, você receberá atualizações diretamente no seu e-mail.

Espero que consiga participar da próxima!

In [ ]:
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura ocorreu de 12 a 16 de maio de 2025. As inscrições foram até 11 de maio de 2025.


In [ ]:
# Exibe a busca
print(f"Busca realizada: {resposta.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in resposta.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(resposta.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['próxima imersão IA Alura Google Gemini']
Páginas utilizadas na resposta: thallesbenicio.com.br, youtube.com



In [ ]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: Buscador de Vagas --- #
##########################################
def agente_busca_vagas(area, data_de_hoje):

  buscador = Agent(
      name="agente_busca_vagas",
      model="gemini-2.0-flash",
      description="agente que busca vagas de empregos na área indicada",
      tools=[google_search],
      instruction="""
      Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
      para recuperar as informações sobre vagas de emprego na área indicada.
      traga também os links das vagas encontradas, eles serão utilizados mais tarde.
      Foque no máximo em 10 vagas relevantes.
      Essas vagas devem ser recentes, de no máximo um mês antes da data de hoje.
      """


  )
  entrada_do_agente_busca_vagas = f"Área: {area}\nData de hoje: {data_de_hoje}"
  # Executa o agente
  vagas = call_agent(buscador, entrada_do_agente_busca_vagas)
  return vagas

In [ ]:
################################################
# --- Agente 2: Filtragem de Vagas --- #
################################################
def agente_de_filtragem(area, lancamentos_buscados):
    filtrador = Agent(
        name="agente_de_filtragem",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um recrutador que faz a triagem de vagas de emprego.
        Com base nas vagas mais recentes e relevantes buscadas, você deve:
        Usar a ferramenta de busca do google (google_search), para fazer uma triagem das vagas encontradas, tendo como base
        as informações encontradas na descrição da vaga, como benefícios, salário, pré-requisitos e apresentar as cinco mais relevantes.

        """,
        description="Agente que faz a triagem das vagas encontradas",
        tools=[google_search]
    )

    entrada_do_agente_de_filtragem = f"Área:{area}\nVagas buscadas: {lancamentos_buscados}"
    # Executa o agente
    selecao_de_vaga = call_agent(filtrador, entrada_do_agente_de_filtragem)
    return selecao_de_vaga

In [ ]:
######################################
# --- Agente 3: Coach de Carreira --- #
######################################
def agente_coach(area, selecao_de_vaga):
    coach = Agent(
        name="agente_coach",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Coach de carreiras, especializado em ajudar pessoas a aplicar para vagas de emprego.
            Você vai receber as cinco vagas filtradas na seleção de vaga e escolher a mais relevante.
            Aponte os pontos mais relevantes e apresente dicas de como ter mais chances de ser contratado.
            Iclua dicas para a entrevista e os principais tópicos que podem ser abordados.
            """,
        description="Agente Coach de carreira"
    )
    entrada_do_agente_redator = f"Área: {area}\nSeleção de vaga: {selecao_de_vaga}"
    # Executa o agente
    rascunho = call_agent(coach, entrada_do_agente_redator)
    return rascunho

In [ ]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(area, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Coach de carreiras meticuloso, especializado em planejar candidaturas a vagas de emprego.
            Revise o rascunho sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para a candidatura!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor plano de aplicação a vagas."
    )
    entrada_do_agente_revisor = f"Área: {area}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Busca de vagas de emprego com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
area = input("❓ Por favor, digite a ÁREA na qual você deseja encontrar uma vaga: ")

# Inserir lógica do sistema de agentes ################################################

if not area:
  print("Você esqueceu de digitar a área!")

else:
  print("Ótimo! Então vamos encontrar sua vaga!")


  vagas_buscadas = agente_busca_vagas(area, data_de_hoje)
  print("\n--- Resultado do agente 1 (Buscador) ---\n")
  display(to_markdown(vagas_buscadas))
  print("--------------------------------------------------------")

  selecao_de_vaga = agente_de_filtragem(area, vagas_buscadas)
  print("\n--- Resultado do agente 2 (Filtrador) ---\n")
  display(to_markdown(selecao_de_vaga))
  print("--------------------------------------------------------")

  # Renomeado a variável 'coached' para 'coach_result' para evitar conflito com a função agente_coach
  coach_result = agente_coach(area, selecao_de_vaga)
  print("\n--- Resultado do agente 3 (Coach) ---\n")
  display(to_markdown(coach_result))


  resultado_final = agente_revisor(area, coach_result)
  print("\n--- Resultado do agente 4 (Revisor) ---\n")
  display(to_markdown(resultado_final))



🚀 Iniciando o Sistema de Busca de vagas de emprego com 4 Agentes 🚀
❓ Por favor, digite a ÁREA na qual você deseja encontrar uma vaga: qa
Ótimo! Então vamos encontrar sua vaga!

--- Resultado do agente 1 (Buscador) ---



> Ok, vou procurar vagas de QA com data de publicação de no máximo um mês atrás (a partir de 17/05/2025).
> 
> 
> Ok, com base na pesquisa, aqui estão algumas vagas de QA que encontrei e informações relevantes:
> 
> **Vagas Encontradas:**
> 
> *   **QA Analista de Testes (Remoto):** Coodesh (há 1 mês) - Remoto. (getVagas)
> *   **QA Sênior (Remoto/Freelance):** ilegra - Projeto até 30.06.2025 (há 1 mês) - Remoto. (getVagas)
> *   **QA Júnior (Remoto):** Devvo Tecnologia (há 1 mês) - Remoto. (getVagas)
> *   **QA (Quality Assurance):** Coodesh (Belo Horizonte, Brasil) - Presencial (há 2 meses, pode estar desatualizada). (getVagas)
> *   **Analista de Suporte TI:** Golden Empregos Contrata (há 1 dia). (getVagas)
> *   **QA na V4 Company:** Campinas/SP (há 4 dias) - Remoto. (getVagas)
> *   **Analista de Qualidade Junior:** Belo Horizonte – MG, Salário: R$ 3.789,00 (Catho)
> *   **Analista de QA Júnior / P+D (Remoto):** BairesDev (Catho)
> *   **Quality Assurance (QA) | Pleno/Sênior:** Kobe Apps - Home office (Indeed)
> *   **Analista de Teste QA - Junior:** (Indeed)
> 
> **Links:**
> 
> *   getVagas: [https://www.getvagas.com.br/vagas/qa](https://www.getvagas.com.br/vagas/qa)
> *   Indeed: Vários links para vagas específicas de QA Júnior, Pleno e Sênior (ver resultados da pesquisa).
> *   Catho: Vários links para vagas específicas de QA Júnior (ver resultados da pesquisa).
> 
> **Informações Salariais:**
> 
> *   **QA Júnior:** Média de R$2.292/mês (Jooble)
> *   **QA Pleno:** Média de R$78.000 anual (Talent.com) ou R$2.300/mês (Jooble)
> *   **QA Engineer:** Média de R$4.535/mês (Jooble)
> 
> **Observações:**
> 
> *   Algumas vagas listadas nos resultados podem já estar fechadas ou desatualizadas (ex: vagas com mais de um mês).
> *   Verifique as datas de publicação das vagas e os requisitos específicos de cada uma nos links fornecidos.
> *   Consulte os sites Indeed e Catho para encontrar mais vagas e aplicar filtros de localização e nível de experiência.
> 
> Espero que estas informações ajudem na sua busca!
> 


--------------------------------------------------------


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-358' coro=<AsyncClient.aclose() done, defined at /usr/local/lib/python3.11/dist-packages/httpx/_client.py:1978> exception=RuntimeError('Event loop is closed')>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/httpx/_client.py", line 1985, in aclose
    await self._transport.aclose()
  File "/usr/local/lib/python3.11/dist-packages/httpx/_transports/default.py", line 406, in aclose
    await self._pool.aclose()
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection_pool.py", line 353, in aclose
    await self._close_connections(closing_connections)
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection_pool.py", line 345, in _close_connections
    await connection.aclose()
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection.py", line 173, in aclose
    await self._connection.aclose()
  File "/usr/local/lib/pyth


--- Resultado do agente 2 (Filtrador) ---



> Com certeza! Para te ajudar a otimizar sua busca por vagas de QA, vou fazer uma triagem das opções listadas, focando nas mais recentes (publicadas no último mês a partir de 17/05/2025) e relevantes, considerando o que geralmente se busca em uma vaga de QA (benefícios, salário, requisitos):
> 
> Para isso, vou realizar algumas pesquisas no Google para complementar as informações que você já levantou.
> 
> 
> Com base nas informações que você forneceu e nas minhas pesquisas, aqui está uma análise das vagas de QA mais relevantes para você, levando em conta a data de publicação, benefícios, salários e requisitos:
> 
> **Top 5 Vagas de QA (Triagem):**
> 
> 1.  **QA Analista de Testes (Remoto) - Coodesh:**
>     *   **Data de publicação:** Há 1 mês (a partir de 17/05/2025).
>     *   **Localização:** Remoto.
>     *   **Benefícios:** Horários Flexíveis, Gympass, Vale flexível, Plano de Saúde, Plano Odontológico, Seguro de Vida.
>     *   **Requisitos:** Técnicas de Testes manuais; Planejamento de Testes a partir de requisitos/histórias; Cypress. SQL e Javascript são habilidades desejáveis.
>     *   **Observações:** Vaga em uma *govtech* com foco em modernizar a relação entre setor público, cidadãos e empresas. Parece uma boa oportunidade para quem quer trabalhar com impacto social. Candidatura feita exclusivamente pela plataforma Coodesh.
> 
> 2.  **QA Sênior (Remoto/Freelance) - ilegra:**
>     *   **Data de publicação:** Há 1 mês (a partir de 17/05/2025).
>     *   **Localização:** Remoto.
>     *   **Benefícios:** A combinar (verificar no processo seletivo).
>     *   **Requisitos:** Inglês fluente (será avaliado); Mínimo de 5 anos de experiência; Experiência internacional comprovada; Disponibilidade para atuação até 30/06/2025; Experiência com criação de planos e casos de teste; Experiência com execução de testes funcionais, de regressão, exploratórios, smoke, etc.; Experiência em projetos internacionais; Experiência com Postman e Splunk; Experiência com Scrum e Kanban; Experiência com JavaScript; Conhecimento em Cypress ou Playwright.
>     *   **Observações:** Projeto com prazo determinado (até 30/06/2025). Exige inglês fluente e experiência sênior comprovada.
> 
> 3.  **QA Júnior (Remoto) - Devvo Tecnologia:**
>     *   **Data de publicação:** Há 1 mês (a partir de 17/05/2025).
>     *   **Localização:** Remoto.
>     *   **Benefícios:** Não especificados (entrar em contato com a empresa).
>     *   **Requisitos:** Experiência no garantia de qualidade de aplicações Web e Mobile; Conhecimento em ferramentas de automação de testes, como Selenium, Cypress ou Appium; Proficiência em metodologias ágeis; Boas habilidades de comunicação e trabalho em equipe.
>     *   **Observações:** Sediada em Brasília. Busca profissionais inovadores e apaixonados por tecnologia.
> 
> 4.  **QA na V4 Company - Campinas/SP:**
>     *   **Data de publicação:** Há 4 dias (a partir de 17/05/2025).
>     *   **Localização:** Híbrido (2 dias remoto e 3 dias presencial em Campinas/SP).
>     *   **Benefícios:** A V4 Company oferece uma série de benefícios, incluindo salários que podem chegar a R$ 5 mil ou mais, dependendo da experiência e da função, plano de saúde e Gympass.
>     *   **Requisitos:** Bacharelado em Ciência da Computação, Engenharia de Software ou área relacionada; Experiência comprovada em testes de software, com ênfase em testes de API e automação; Conhecimento sólido de princípios e metodologias de teste de software; Proficiência em pelo menos uma linguagem de programação relevante para automação de testes (por exemplo, Python, Java, JavaScript); Experiência com ferramentas de automação de testes de API (por exemplo, Postman, REST Assured, Karate); Familiaridade com protocolos de API (REST, SOAP).
>     *   **Observações:** A V4 Company está em expansão e oferece um ambiente de trabalho vibrante.
> 
> 5.  **Analista de Qualidade Junior - Belo Horizonte – MG:**
>     *   **Data de publicação:** Indeterminado (Catho).
>     *   **Localização:** Belo Horizonte – MG (Presencial).
>     *   **Salário:** R$ 3.789,00.
>     *   **Benefícios:** Vale-alimentação (R$ 225,00), Vale-refeição (R$ 14,20 por dia), Vale-transporte, Programa Bem-Estar (Telemedicina, Total Pass, Vidalink Med, Vidalink Nutri, Vidalink Up), Assistência médica (Unimed), Assistência odontológica e parcerias com faculdades e o Sesc.
>     *   **Requisitos:** Ensino superior completo ou em andamento em Administração, Engenharia, Gestão da Qualidade ou áreas afins; Habilidade no uso e interpretação de relatórios e indicadores; Disponibilidade para viagens (eventuais ou frequentes, conforme necessidade); Conhecimento em Pacote Office intermediário (ênfase em Excel). Desejável: conhecimento em ferramentas da qualidade (PDCA, 5W2H).
>     *   **Observações:** Oportunidade para quem busca uma vaga presencial em BH com bons benefícios.
> 
> **Considerações Adicionais:**
> 
> *   **Salários:** Os salários para QA variam bastante dependendo do nível de experiência e da empresa. QA Júnior pode ganhar em média R$2.292/mês, enquanto QA Pleno pode chegar a R$78.000 anual.
> *   **Outras Vagas:** Explore os sites Indeed e Catho para encontrar mais vagas e aplicar filtros de localização e nível de experiência.
> *   **Atualização:** Verifique sempre a data de publicação das vagas e os requisitos específicos de cada uma nos links fornecidos, pois algumas informações podem estar desatualizadas.
> 
> Espero que esta triagem detalhada te ajude a focar nas vagas mais promissoras para sua carreira em QA!


--------------------------------------------------------

--- Resultado do agente 3 (Coach) ---



> Com prazer! Analisando as 5 vagas de QA que você listou, a que parece mais promissora, considerando um equilíbrio entre benefícios, desafios e potencial de crescimento, é a de:
> 
> **QA Analista de Testes (Remoto) - Coodesh**
> 
> **Por que essa vaga se destaca:**
> 
> *   **Remoto:** Flexibilidade para trabalhar de qualquer lugar.
> *   **Benefícios:** Pacote completo (Vale flexível, Plano de Saúde, Plano Odontológico, Seguro de Vida, Gympass).
> *   **Requisitos:** Foco em habilidades práticas (Testes manuais, Planejamento de Testes, Cypress), com SQL e Javascript como diferenciais.
> *   **Govtech:** Oportunidade de trabalhar em projetos com impacto social.
> *   **Plataforma Coodesh:** Processo seletivo otimizado e focado em desenvolvedores.
> 
> **Estratégias para aumentar suas chances de ser contratado:**
> 
> 1.  **Destaque suas habilidades com Cypress:** Se você tem experiência com Cypress, essa é a hora de brilhar. Prepare exemplos de projetos ou testes que você desenvolveu usando essa ferramenta para mostrar seu conhecimento prático.
> 2.  **Mostre sua capacidade de planejamento:** Detalhe sua experiência em planejar testes a partir de requisitos e histórias. Explique como você prioriza os testes, define a cobertura e garante a qualidade do software.
> 3.  **Valorize suas habilidades em SQL e Javascript:** Mesmo que sejam habilidades desejáveis, mostre que você as possui e como elas podem agregar valor ao time de QA.
> 4.  **Prepare-se para a plataforma Coodesh:** A Coodesh tem um formato próprio de avaliação. Pesquise sobre a plataforma, familiarize-se com os tipos de testes e desafios que eles propõem e treine suas habilidades técnicas.
> 5.  **Pesquise sobre a empresa:** Descubra mais sobre a Coodesh, seus projetos, sua cultura e seus valores. Mostre que você está alinhado com a missão da empresa e que você quer fazer a diferença no setor público.
> 
> **Dicas para a entrevista:**
> 
> *   **Prepare-se para perguntas técnicas:** Espere perguntas sobre testes manuais, planejamento de testes, Cypress, SQL, Javascript e metodologias ágeis.
> *   **Destaque sua experiência com projetos de impacto social:** Se você já trabalhou em projetos com foco em impacto social, conte sobre sua experiência e como você contribuiu para o sucesso desses projetos.
> *   **Mostre suas habilidades de comunicação e trabalho em equipe:** Explique como você se comunica com o time de desenvolvimento, como você colabora para resolver problemas e como você se adapta a diferentes contextos.
> *   **Faça perguntas relevantes:** Mostre seu interesse pela vaga e pela empresa fazendo perguntas sobre os projetos, os desafios, as tecnologias e as oportunidades de crescimento.
> *   **Seja você mesmo:** Mostre sua paixão por QA, sua vontade de aprender e sua capacidade de fazer a diferença.
> 
> **Tópicos que podem ser abordados na entrevista:**
> 
> *   **Experiência com testes manuais e automatizados:** Detalhe sua experiência com diferentes tipos de testes, como testes funcionais, de regressão, de performance e de segurança.
> *   **Conhecimento de metodologias ágeis:** Explique como você trabalha em um ambiente ágil, como você participa das cerimônias do Scrum ou Kanban e como você contribui para a melhoria contínua do processo.
> *   **Habilidade de identificar e reportar bugs:** Mostre sua capacidade de encontrar bugs, documentá-los de forma clara e objetiva e acompanhar sua correção.
> *   **Experiência com ferramentas de teste:** Mencione as ferramentas que você já utilizou, como Jira, TestRail, Selenium, Appium, etc.
> *   **Conhecimento de métricas de qualidade:** Explique como você mede a qualidade do software, como você define metas e como você acompanha os resultados.
> 
> Lembre-se de adaptar essas dicas à sua experiência e às suas habilidades. Boa sorte na sua candidatura!



--- Resultado do agente 4 (Revisor) ---



> O rascunho está ótimo e pronto para a candidatura!
